In [1]:
import io 
import pandas as pd 
import pyarrow as pa   # modern backend for Numpy

# Jupyter

In [2]:
%timeit x = range(1000) 
# line magic
# %timeit line magic can be used to time how long it takes to run a line of code

40.1 ns ± 1.65 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [3]:
%%timeit
# magic cell works on the entire cell.
x = range(1000)
total = 0
for i in x:
    total += i
mean = total / len(x)

22.5 μs ± 648 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Some of the most useful magics are:

`%matplotlib inline`
This line magic will display matplotlib plots inline in the notebook. (This is the default behavior in JupyterLab and is not needed there.)

`%timeit`
This line magic will time how long it takes to run a line of code.

`%%timeit`
This cell magic will time how long it takes to run a cell of code.

`%debug`
This line magic will start the interactive debugger.

`%pdb`
This line magic will enable the interactive debugger.

`%%html`
This cell magic will render the cell as HTML.

`%%javascript`
This cell magic will run the cell’s contents as JavaScript code.

`%%writefile`
This cell magic will write the cell’s contents to a file.

# Pandas

|Data Type	| SQL	| Python	| Pandas 2	|Pandas 1 |
|-----------|------|-----------|----------|----------|
|Integer |	INT|	int|	'int64[pyarrow]'|	'int64' |
|Float	| FLOAT |	float|	'float64[pyarrow]'	|'float64' |
|String|	VARCHAR	|str|	pd.ArrowDType( pa.String()) |	object|str |
|Date	|DATE|	datetime|	'timestamp[ns] [pyarrow]'|	'datetime64[ns]' |
|Categorical|	N/A|	N/A|	'dictionary' |	'category' |
|Boolean|	BOOLEAN|	bool|	'bool[pyarrow]'|	bool |


# page_number: 16,23,39,40

In [4]:
print(pd.Series([1, 99, 127],dtype='int8'))
print(pd.Series([1,99,127],dtype='int8[pyarrow]'))


print(pd.Series([2**31, 2**63, 2**100], )   ) # object dtype
print(pd.Series([2**31, 2**63, 2**100], dtype='double') )  # explicit mention 'double' or 'float64'

# PyArrow doesn't gracefully turn into python Obj instead it'll throw error
# print(pd.Series([2**31, 2**63, 2**100], dtype='int64[pyarrow]') )  # python int too large to convert to C long


try:
    print(pd.Series([2**31, 2**63, 2**100],dtype=pd.ArrowDtype(pa.float64())) )  # explicit mention
except Exception as e:
    print(str(e))


# Note that the NumPy-backed series has a type of object. 
# Because the numbers are larger than an int64, NumPy will store the values but store them as Python objects. 
# This will make the code run slower and use more memory.


0      1
1     99
2    127
dtype: int8
0      1
1     99
2    127
dtype: int8[pyarrow]
0                         2147483648
1                9223372036854775808
2    1267650600228229401496703205376
dtype: object
0    2.147484e+09
1    9.223372e+18
2    1.267651e+30
dtype: float64
Python int too large to convert to C long


In [5]:
print(pd.Series([None, 1, -45]))  # NaN backed series is float64.
print(pd.Series([None, 1, -45],dtype='int8[pyarrow]'))   # pyarrow supports

0     NaN
1     1.0
2   -45.0
dtype: float64
0    <NA>
1       1
2     -45
dtype: int8[pyarrow]


In [6]:
print(pd.Series(['My name is Muthu', 'I like pandas', 'I like programming',None]))  # object
print(pd.Series(['My name is Muthu', 'I like pandas', 'I like programming',None],dtype='category'))  # category
print(pd.Series(['My name is Muthu', 'I like pandas', 'I like programming',None],dtype=pd.ArrowDtype(pa.string())))

0      My name is Muthu
1         I like pandas
2    I like programming
3                  None
dtype: object
0      My name is Muthu
1         I like pandas
2    I like programming
3                   NaN
dtype: category
Categories (3, object): ['I like pandas', 'I like programming', 'My name is Muthu']
0      My name is Muthu
1         I like pandas
2    I like programming
3                  <NA>
dtype: string[pyarrow]


In [7]:
import datetime as dt
dt_list = [dt.datetime(2020, 1, 1, 4, 30 ), dt.datetime(2020, 1, 2), dt.datetime(2020, 1, 3)]
string_dates = ['2020-01-01 04:30:00', '2020-01-02 00:00:00', '2020-01-03 00:00:00']
string_dates_missing = ['2020-01-01 4:30', None, '2020-01-03']
epoch_dates = [1577836800, 1577923200, 1578009600]

In [8]:
pd.Series(string_dates, dtype='datetime64[ns]')

0   2020-01-01 04:30:00
1   2020-01-02 00:00:00
2   2020-01-03 00:00:00
dtype: datetime64[ns]

In [9]:
pd.Series(epoch_dates, dtype='datetime64[s]') 

0   2020-01-01
1   2020-01-02
2   2020-01-03
dtype: datetime64[s]

In [10]:
pd.Series(dt_list, dtype='timestamp[ns][pyarrow]')

0    2020-01-01 04:30:00
1    2020-01-02 00:00:00
2    2020-01-03 00:00:00
dtype: timestamp[ns][pyarrow]

In [11]:
pd.Series(epoch_dates).astype('timestamp[s][pyarrow]')

0    2020-01-01 00:00:00
1    2020-01-02 00:00:00
2    2020-01-03 00:00:00
dtype: timestamp[s][pyarrow]

In [12]:
pd.options.compute.use_numba

False

In [13]:
url = 'https://github.com/mattharrison/datasets/raw/master/data/siena2018-pres.csv'
df = pd.read_csv(url, index_col=0, dtype_backend='pyarrow')

In [14]:
def tweak_siena_pres(df):
     def int64_to_uint8(df_):
         cols = df_.select_dtypes('int64')
         return (df_
                 .astype({col:'uint8[pyarrow]' for col in cols}))

     return (df
      .rename(columns={'Seq.':'Seq'})    # 1
      .rename(columns={k:v.replace(' ', '_') for k,v in
         {'Bg': 'Background',
          'PL': 'Party leadership', 'CAb': 'Communication ability',
          'RC': 'Relations with Congress', 'CAp': 'Court appointments',
          'HE': 'Handling of economy', 'L': 'Luck',
          'AC': 'Ability to compromise', 'WR': 'Willing to take risks',
          'EAp': 'Executive appointments', 'OA': 'Overall ability',
          'Im': 'Imagination', 'DA': 'Domestic accomplishments',
          'Int': 'Integrity', 'EAb': 'Executive ability',
          'FPA': 'Foreign policy accomplishments',
          'LA': 'Leadership ability',
          'IQ': 'Intelligence', 'AM': 'Avoid crucial mistakes',
          'EV': "Experts' view", 'O': 'Overall'}.items()})
      .astype({'Party':'category'})  # 2
      .pipe(int64_to_uint8)  # 3
      .assign(Average_rank=lambda df_:(df_.select_dtypes('uint8') # 4
         .sum(axis=1).rank(method='dense').astype('uint8[pyarrow]')),
              Quartile=lambda df_:pd.qcut(df_.Average_rank, 4,
                   labels='1st 2nd 3rd 4th'.split())
             )
     )
pres = tweak_siena_pres(df=df)

In [15]:
pres.select_dtypes('number').agg([
    'count',
    'size',
    'sum',
    lambda col: col.loc[1]
])

,Background,Imagination,Integrity,Intelligence,Luck,Willing_to_take_risks,Ability_to_compromise,Executive_ability,Leadership_ability,Communication_ability,...,Relations_with_Congress,Court_appointments,Handling_of_economy,Executive_appointments,Domestic_accomplishments,Foreign_policy_accomplishments,Avoid_crucial_mistakes,Experts'_view,Overall,Average_rank
count,44,44,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,44,44
size,44,44,44,44,44,44,44,44,44,44,...,44,44,44,44,44,44,44,44,44,44
sum,968,957,990,990,990,953,968,978,990,990,...,979,990,990,990,990,990,990,990,990,990
<lambda>,7,7,1,10,1,6,2,2,1,11,...,1,1,1,1,2,2,1,2,1,1


In [16]:
pres.select_dtypes('number').agg({
    'Luck':['count','size'],
    'Overall':['count','max']
})

,Luck,Overall
count,44.0,44.0
size,44.0,NaN
max,NaN,44.0


In [17]:
pres.select_dtypes('number').agg(
    Intelligence_count = ('Intelligence','count'),
    Intelligenve_size = ("Intelligence",'size')
)

,Intelligence
Intelligence_count,44
Intelligenve_size,44


In [18]:
billing_data = '''cancel_date,period_start,start_date,end_date,rev,sum_payments
12/1/2019,1/1/2020,12/15/2019,5/15/2020,999,50
,1/1/2020,12/15/2019,5/15/2020,999,50
,1/1/2020,12/15/2019,5/15/2020,999,1950
1/20/2020,1/1/2020,12/15/2019,5/15/2020,499,0
,1/1/2020,12/24/2019,5/24/2020,699,100
,1/1/2020,11/29/2019,4/29/2020,799,250
,1/1/2020,1/15/2020,4/29/2020,799,250'''
bill_df = pd.read_csv(io.StringIO(billing_data),dtype_backend='pyarrow',parse_dates=['cancel_date','period_start','start_date','end_date'])

def tweak_bill202(df_:pd.DataFrame):
    return df_.assign(
        cancel_date = pd.to_datetime( df_.cancel_date.replace('<NA>',''),format=r'%m/%d/%Y' )
    )
tweak_bill202(bill_df)

,cancel_date,period_start,start_date,end_date,rev,sum_payments
0,2019-12-01,2020-01-01,2019-12-15,2020-05-15,999,50
1,NaT,2020-01-01,2019-12-15,2020-05-15,999,50
2,NaT,2020-01-01,2019-12-15,2020-05-15,999,1950
3,2020-01-20,2020-01-01,2019-12-15,2020-05-15,499,0
4,NaT,2020-01-01,2019-12-24,2020-05-24,699,100
5,NaT,2020-01-01,2019-11-29,2020-04-29,799,250
6,NaT,2020-01-01,2020-01-15,2020-04-29,799,250
